# Loading OME-Zarrs in Python

In this notebook we will load OME-Zarr files from the web using different libraries.

We'll visualize the images using `napari` and web viewers.

## URL of the OME-Zarr file

OME-Zarr is a web compatible format, so we can load it directly from a URL.

In [13]:
url = "https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.4/idr0062A/6001240.zarr"

## Using the `ome-zarr-py` library

In [14]:
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader

# read the image data
reader = Reader(parse_url(url))

# nodes may include images, labels etc
nodes = list(reader())

# first node will be the image pixel data
image_node = nodes[0]

dask_data = image_node.data

/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [3]:
# We can view this in napari
# NB: image axes are CZYX: split channels by C axis=0

import napari

viewer = napari.view_image(dask_data, channel_axis=0)

## Using the ngff-zarr library

In [4]:
import ngff_zarr as nz

multiscales = nz.from_ngff_zarr(url)
multiscales

/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


Multiscales(images=[NgffImage(data=dask.array<from-zarr, shape=(2, 236, 275, 271), dtype=uint16, chunksize=(1, 1, 275, 271), chunktype=numpy.ndarray>, dims=['c', 'z', 'y', 'x'], scale={'c': 1.0, 'z': 0.5002025531914894, 'y': 0.3603981534640209, 'x': 0.3603981534640209}, translation={'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0}, name='image', axes_units={'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}, computed_callbacks=[]), NgffImage(data=dask.array<from-zarr, shape=(2, 236, 137, 135), dtype=uint16, chunksize=(1, 1, 137, 135), chunktype=numpy.ndarray>, dims=['c', 'z', 'y', 'x'], scale={'c': 1.0, 'z': 0.5002025531914894, 'y': 0.7207963069280418, 'x': 0.7207963069280418}, translation={'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0}, name='image', axes_units={'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}, computed_callbacks=[]), NgffImage(data=dask.array<from-zarr, shape=(2, 236, 68, 67), dtype=uint16, chunksize=(1, 1, 68, 67), chunktype=numpy.ndarray>, dims=

In [5]:
import dask.array as da
da.from_zarr(url+'/labels/0/0')

dask.array<from-zarr, shape=(1, 236, 275, 271), dtype=int8, chunksize=(1, 59, 69, 136), chunktype=numpy.ndarray>

In [13]:
multiscales.__dict__

{'images': [NgffImage(data=dask.array<from-zarr, shape=(2, 236, 275, 271), dtype=uint16, chunksize=(1, 1, 275, 271), chunktype=numpy.ndarray>, dims=['c', 'z', 'y', 'x'], scale={'c': 1.0, 'z': 0.5002025531914894, 'y': 0.3603981534640209, 'x': 0.3603981534640209}, translation={'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0}, name='image', axes_units={'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}, computed_callbacks=[]),
  NgffImage(data=dask.array<from-zarr, shape=(2, 236, 137, 135), dtype=uint16, chunksize=(1, 1, 137, 135), chunktype=numpy.ndarray>, dims=['c', 'z', 'y', 'x'], scale={'c': 1.0, 'z': 0.5002025531914894, 'y': 0.7207963069280418, 'x': 0.7207963069280418}, translation={'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0}, name='image', axes_units={'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}, computed_callbacks=[]),
  NgffImage(data=dask.array<from-zarr, shape=(2, 236, 68, 67), dtype=uint16, chunksize=(1, 1, 68, 67), chunktype=numpy.ndarray>, dims=['c'

What does the returned `multiscales` object look like? How do we access the data?

In [6]:
import pprint
pprint.pprint(multiscales.__dict__)

{'chunks': None,
 'images': [NgffImage(data=dask.array<from-zarr, shape=(2, 236, 275, 271), dtype=uint16, chunksize=(1, 1, 275, 271), chunktype=numpy.ndarray>,
                      dims=['c', 'z', 'y', 'x'],
                      scale={'c': 1.0,
                             'x': 0.3603981534640209,
                             'y': 0.3603981534640209,
                             'z': 0.5002025531914894},
                      translation={'c': 0.0, 'x': 0.0, 'y': 0.0, 'z': 0.0},
                      name='image',
                      axes_units={'c': None,
                                  'x': 'micrometer',
                                  'y': 'micrometer',
                                  'z': 'micrometer'},
                      computed_callbacks=[]),
            NgffImage(data=dask.array<from-zarr, shape=(2, 236, 137, 135), dtype=uint16, chunksize=(1, 1, 137, 135), chunktype=numpy.ndarray>,
                      dims=['c', 'z', 'y', 'x'],
                      scale={'c': 

In [7]:
# here's how we can access the data

# this is the first image in the multiscale pyramidal representation
multiscales.images[0].data

dask.array<from-zarr, shape=(2, 236, 275, 271), dtype=uint16, chunksize=(1, 1, 275, 271), chunktype=numpy.ndarray>

Let's show the highest resolution in napari.

In [19]:
viewer = napari.view_image(
    multiscales.images[0].data,
    channel_axis=0)

Let's show the multiscale pyramidal image in napari.

In [12]:
viewer = napari.view_image(
    [image.data for image in multiscales.images],
    channel_axis=0)

## Open in napari directly

Using the `napari-ome-zarr` plugin.

In [16]:
viewer = napari.Viewer()
viewer.open(
    # "https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.4/idr0101A/13457537.zarr",
    "https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.4/idr0062A/6001240.zarr",
    plugin="napari-ome-zarr")

/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/napari/utils/colormaps/colormap.py:435: UserWarning: color_dict did not provide a default color. Missing keys will be transparent. To provide a default color, use the key `None`, or provide a defaultdict instance.
  warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


[<Image layer 'LaminB1' at 0x295d9ac50>,
 <Image layer 'Dapi' at 0x2a12f8dd0>,
 <Labels layer '0' at 0x2a491ac50>]

## Load in web viewers

There's a website to give an overview over a OME-Zarr file (and validate it):

`https://ome.github.io/ome-ngff-validator/?source=`

The link just needs to be completed with the URL of the OME-Zarr file. For the example above, the link would be:

https://ome.github.io/ome-ngff-validator/?source=https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.4/idr0062A/6001240.zarr

It also displays links to load the OME-Zarr in web viewers like `neuroglancer` or `vizarr`.

## Save as OME-Zarr locally

Let's save the OME-Zarr file locally and inspect the file structure.

In [21]:
multiscales = nz.from_ngff_zarr(url)

nz.to_ngff_zarr(
    './test_image.ome.zarr',
    multiscales,
)

/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/dask/array/utils.py:104: RuntimeWarning: invalid value encountered in cast
  meta = meta.astype(dtype)


In [ ]:
# view it in napari

viewer = napari.Viewer()
viewer.open(
    './test_image.ome.zarr',
    plugin="napari-ome-zarr")

/Users/malbert/miniconda3/envs/day3/lib/python3.11/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


[<Image layer 'LaminB1' at 0x2925f8550>, <Image layer 'Dapi' at 0x296c323d0>]

Or we can save any array to a file (simple mode).

In [35]:
import numpy as np

image = np.random.randint(0, 255, (10, 10, 10), dtype=np.uint8)

ngff_image = nz.ngff_image.NgffImage(
    data=image,
    dims=['z', 'y', 'x'],
    scale={'z': 1., 'y': 0.5, 'x': 0.5},
    translation={'z': 0., 'y': 0., 'x': 0.},
)

multiscales = nz.to_multiscales(
    ngff_image,
    [2, 4]
)

nz.to_ngff_zarr(
    './test_image_np.ome.zarr',
    multiscales,
)

## Load local OME-Zarr in web viewers

In order to access our local file over the web, we need to serve the file over HTTP.

In [35]:
import os
os.system('ome_zarr view ./test_image_np.ome.zarr')

# This cell runs until it is interrupted (jupyter menu Kernel > Interrupt)

Serving HTTP on :: port 8000 (http://[::]:8000/) ...


::1 - - [13/May/2025 00:09:25] code 501, message Unsupported method ('OPTIONS')
::1 - - [13/May/2025 00:09:25] "OPTIONS /test_image_np.ome.zarr/zarr.json HTTP/1.1" 501 -
::1 - - [13/May/2025 00:09:25] code 404, message File not found
::1 - - [13/May/2025 00:09:25] "GET /test_image_np.ome.zarr/zarr.json HTTP/1.1" 404 -
::1 - - [13/May/2025 00:09:25] code 501, message Unsupported method ('OPTIONS')
::1 - - [13/May/2025 00:09:25] "OPTIONS /test_image_np.ome.zarr/.zattrs HTTP/1.1" 501 -
::1 - - [13/May/2025 00:09:25] "GET /test_image_np.ome.zarr/.zattrs HTTP/1.1" 304 -
::1 - - [13/May/2025 00:09:25] code 501, message Unsupported method ('OPTIONS')
::1 - - [13/May/2025 00:09:25] "OPTIONS /test_image_np.ome.zarr/labels/.zattrs HTTP/1.1" 501 -
::1 - - [13/May/2025 00:09:25] code 404, message File not found
::1 - - [13/May/2025 00:09:25] "GET /test_image_np.ome.zarr/labels/.zattrs HTTP/1.1" 404 -
::1 - - [13/May/2025 00:09:25] code 501, message Unsupported method ('OPTIONS')
::1 - - [13/May/20


Keyboard interrupt received, exiting.


0

In [ ]:
# import dask.array as da
# arr = da.from_zarr("http://localhost:8000/test_image_np.ome.zarr/scale0/image")

## Writing

In [ ]:
import numpy as np
import zarr

from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

path = "test_ngff_image.zarr"

size_xy = 128
size_z = 10
rng = np.random.default_rng(0)
data = rng.poisson(lam=10, size=(size_z, size_xy, size_xy)).astype(np.uint8)

# write the image data
store = parse_url(path, mode="w").store
root = zarr.group(store=store)
write_image(image=data, group=root, axes="zyx", storage_options=dict(chunks=(1, size_xy, size_xy)))